In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("StressSleepAnalysis").getOrCreate()

hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set("fs.s3a.endpoint", "http://minio:9000")
hadoopConf.set("fs.s3a.access.key", "admin")
hadoopConf.set("fs.s3a.secret.key", "admin123")
hadoopConf.set("fs.s3a.path.style.access", "true")
hadoopConf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

In [2]:
df = spark.read.csv("s3a://datalake/raw/stress_sleep.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)

Py4JJavaError: An error occurred while calling o28.csv.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNe

In [ ]:
df.groupBy("Have you ever been diagnosed with a mental health condition by a professional (doctor, therapist, etc.)?").count().show()

In [ ]:
df_clean = df.dropna(subset=[
    "Gender", 
    "Age", 
    "How many hours of actual sleep did you get on an average for the past month? (maybe different from the number of hours spent in bed)"
])

In [ ]:
df_clean.write.mode("overwrite").parquet("s3a://datalake/trusted/stress_sleep_clean.parquet")